In [9]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [10]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        OneHotDegree(max_degree=8,cat=False)
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 9
Number of classes: 2

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 9], y=[1])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [11]:
data.betti_numbers

[1, 0, 1]

In [12]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")


Number of training graphs: 562
Number of test graphs: 150


In [13]:
train_loader = DataLoader(train_dataset,batch_size=10)#,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset,batch_size=10)


for batch in train_loader:
    break

batch.y

tensor([0, 0, 0, 1, 0, 0, 1, 0, 0, 0])

In [14]:
from operator import concat
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TAGConv,TransformerConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = TransformerConv(dataset.num_node_features, hidden_channels)
        self.conv2 = TransformerConv(hidden_channels, hidden_channels)
        self.conv3 = TransformerConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 3)

    def forward(self, batch):
        # 1. Obtain node embeddings
        x = self.conv1(batch.x, batch.edge_index)
        x = x.relu()
        x = self.conv2(x, batch.edge_index)
        x = x.relu()
        x = self.conv3(x, batch.edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch.batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


model = GCN(hidden_channels=64)
print(model(batch))


tensor([[ 0.0788, -0.0744, -0.0329],
        [ 0.1500, -0.0435, -0.1150],
        [ 0.1908,  0.1661, -0.0821],
        [ 0.1692,  0.0347, -0.2018],
        [-0.1191, -0.0769,  0.0184],
        [ 0.2564,  0.0817, -0.2506],
        [-0.0386, -0.1109, -0.0483],
        [ 0.2014, -0.0658,  0.0422],
        [ 0.2608, -0.0205, -0.1961],
        [ 0.1721,  0.1232, -0.2513]], grad_fn=<AddmmBackward0>)


In [15]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass.
        loss = criterion(out, torch.tensor(data.betti_numbers,dtype=torch.float))  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    losses = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)
        
        losses += criterion(out, torch.tensor(data.betti_numbers,dtype=torch.float))
    return losses


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 14.3012, Test Acc: 3.9595
Epoch: 002, Train Acc: 7.9209, Test Acc: 2.2424
Epoch: 003, Train Acc: 7.3249, Test Acc: 2.1451
Epoch: 004, Train Acc: 7.0863, Test Acc: 2.0972
Epoch: 005, Train Acc: 6.5136, Test Acc: 1.9514
Epoch: 006, Train Acc: 5.7232, Test Acc: 1.6731
Epoch: 007, Train Acc: 5.6288, Test Acc: 1.6629
Epoch: 008, Train Acc: 5.3735, Test Acc: 1.6019
Epoch: 009, Train Acc: 5.5434, Test Acc: 1.6216
Epoch: 010, Train Acc: 5.0546, Test Acc: 1.4855
Epoch: 011, Train Acc: 5.0977, Test Acc: 1.4528
Epoch: 012, Train Acc: 5.1413, Test Acc: 1.5387
Epoch: 013, Train Acc: 5.0282, Test Acc: 1.4862
Epoch: 014, Train Acc: 4.5230, Test Acc: 1.3564
Epoch: 015, Train Acc: 4.5798, Test Acc: 1.3817
Epoch: 016, Train Acc: 4.7982, Test Acc: 1.4428
Epoch: 017, Train Acc: 5.1819, Test Acc: 1.5367
Epoch: 018, Train Acc: 5.0257, Test Acc: 1.5190
Epoch: 019, Train Acc: 4.4559, Test Acc: 1.3445
Epoch: 020, Train Acc: 4.2240, Test Acc: 1.2602
Epoch: 021, Train Acc: 4.5640, Test Acc

In [16]:
model.eval()
losses = 0
y_hat = []
y = []
for data in test_loader:  # Iterate in batches over the training/test dataset.
    out = model(data)
    y_hat.append(out.round().long())
    y.append(torch.tensor(data.betti_numbers))

y_hat = torch.vstack(y_hat)
y = torch.vstack(y)


incorrect = torch.count_nonzero(y-y_hat!=0).item()
correct = torch.count_nonzero(y-y_hat==0).item()

print("predicted incorrect",correct)
print("predicted incorrect",incorrect)
print("percentage correct", correct / (correct + incorrect))


predicted incorrect 395
predicted incorrect 55
percentage correct 0.8777777777777778
